In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import sys, os
proj = os.path.dirname(os.getcwd())
if proj not in sys.path:
    sys.path.insert(0, proj)
sys.path

['/Volumes/Projects/Evidently/record-thing',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python311.zip',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/lib-dynload',
 '',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages']

In [4]:
# Device independent code
import torch

device = "cpu" 
if torch.cuda.is_available():
    device = "cuda"
# if torch.backends.mps.is_available:
#     device = torch.device("mps")
torch.device(device)
torch.set_default_device(device=device)

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [16]:
!docker compose up -d

/Users/henrikvendelbo/.zshenv:.:1: no such file or directory: /Users/henrikvendelbo/.cargo/env
[+] Running 1/0
 ✔ Container rqlite_server  Running                                        0.0s 


In [ ]:
from dataset.db import init_db_sqlite
connection = init_db_sqlite(disconnect=False)
connection

# UPC/QR lookup

If the scanner recognises a barcode or QR code the product number should be looked up.
If multiple matches are found the category of product should be used to pick the most likely match.
The user is asked to confirm the match YES/NO.

In [ ]:
from dataset.product import lookup_product
from dataset.ingestion import Ingestor

upc = '0885909950805'
with Ingestor(connection.cursor(), device=device) as ingestor:

    result = lookup_product(upc, ingestor=ingestor, save=True)
    if result:
        print(result)

        # match with product["items"][0]["images"] product similarity with pictures of the recording.
        # print(number_matches, upc, item)

{'upc': '885909950805', 'asin': 'B00NQGOZV0', 'elid': '195389220744', 'brand': 'Apple', 'model': 'MG5A2LL/A', 'color': 'Space Gray', 'category': 'Electronics > Communications > Telephony > Mobile Phones > Unlocked Mobile Phones', 'title': 'Apple iPhone 6, Space Gray, 64 GB (T-Mobile)', 'description': 'Display   Retina HD display 1334-by-750-pixel resolution at 326 ppi 1400:1 contrast ratio (typical) 500 cd/m2 max brightness (typical) Full sRGB standard Dual-domain pixels for wider viewing angles Fingerprint-resistant oleophobic coating on front Support for display of multiple languages and characters simultaneously Display Zoom iSight Camera Autofocus with Focus Pixels f/2.2 aperture True Tone flashHybrid IR filter Backside illumination sensor Sapphire crystal lens cover Auto image stabilization Auto HD...', 'upc_link': 'https://www.upcitemdb.com/norob/alink/?id=v2p2z213v2x28474u2&tid=1&seq=1733392205&plt=cf7aa6eb9640d9edc6d6eda877e8e47a', 'domain': 'newegg.com', 'images': ['http://img

In [ ]:
from typing import Sequence
import torch
import requests

def generate_embedding(image: bytes) -> torch.Tensor:
    # dino.
    pass

def get_images(image_urls: Sequence[str]) -> Sequence[bytes]:
    images: Sequence[bytes] = [requests.get(url).content for url in image_urls]
    embeddings: Sequence[torch.Tensor] = [generate_embedding(image) for image in images]